---
description: Examplel on how to use Langfuse Prompt Management in Langchain applications in Python
---

# Example: Langfuse Prompt Management with Langchain in Python

[Langfuse Prompt Management](https://langfuse.com/docs/prompts) helps to version control and manage prompts collaboratively in one place. This example demostrates how to use prompts managed in Langfuse to run Langchain applications.

## Setup

In [ ]:
%pip install langfuse langchain langchain-openai openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.2 MB/s eta 0:00:00


In [ ]:
import os

# get keys for your project from https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""

# your openai key
os.environ["OPENAI_API_KEY"] = ""

# if you do not use Langfuse Cloud
# os.environ["LANGFUSE_HOST"] = "http://localhost:3000"

In [ ]:
from langfuse import Langfuse
from langfuse.callback import CallbackHandler

langfuse = Langfuse()

langfuse_handler = CallbackHandler()

# optional, verify that Langfuse is configured correctly
print(langfuse.auth_check())
print(langfuse_handler.auth_check())

True
True


## Create Prompt in Langfuse

- `Name` that identifies the prompt in Langfuse Prompt Management
- Prompt with prompt template incl. `{{input variables}}`
- Config including `model_name` and `temperature`
- `is_active` to immediately use prompt

In [ ]:
langfuse.create_prompt(
    name="event-planner",
    prompt=
    "Plan an event titled {{Event Name}}. The event will be about: {{Event Description}}. "
    "The event will be held in {{Location}} on {{Date}}. "
    "Consider the following factors: audience, budget, venue, catering options, and entertainment. "
    "Provide a detailed plan including potential vendors and logistics.",
    config={
        "model":"gpt-3.5-turbo-1106",
        "temperature": 0,
    },
    is_active=True
)

## Get current prompt version from Langfuse

In [53]:
# Get current production version of prompt
langfuse_prompt = langfuse.get_prompt("event-planner")
print(langfuse_prompt.prompt)

Plan an event titled {{Event Name}}. The event will be about: {{Event Description}}. The event will be held in {{Location}} on {{Date}}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.


## Extract Langfuse prompt template prompt and config

As you saw earlier, Langfuse declares input variables of prompt templates with double brackets (`{{input variable}}`). Langchain, on the other side, uses single brackets for declaring input variables in PromptTemplates (`{input variable}`). Hence, we need to edit the template before declaring the Langchain template.

In [54]:
import re

# Replace double brackets with single brackets
single_bracket_prompt = re.sub(r'\{\{(.*?)\}\}', r'{\1}', langfuse_prompt.prompt)
print(single_bracket_prompt)

Plan an event titled {Event Name}. The event will be about: {Event Description}. The event will be held in {Location} on {Date}. Consider the following factors: audience, budget, venue, catering options, and entertainment. Provide a detailed plan including potential vendors and logistics.


The other configurations can be extracted straightforward

In [ ]:
model = langfuse_prompt.config["model"]
temperature = str(langfuse_prompt.config["temperature"])
print(f"Prompt model configurations\nModel: {model}\nTemperature: {temperature}")

Prompt model configurations
Model: gpt-3.5-turbo-1106
Temperature: 0


## Build Langchain application

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

langchain_prompt = ChatPromptTemplate.from_template(single_bracket_prompt)
model = ChatOpenAI(model=model, temperature=temperature)

chain = langchain_prompt | model

## Use Langchain chain with Langfuse callback handler

In [ ]:
response = chain.invoke(
    input={
      "Event Name": "Wedding",
      "Event Description": "The wedding of Julia and Alex, a charming couple "
      " who share a love for art and nature. This special day will celebrate "
      "their journey together with a blend of traditional and contemporary elements, "
      "reflecting their unique personalities.",
      "Location": "Central Park, New York City",
      "Date": "June 5, 2024"
    },
    config={
        "callbacks":[langfuse_handler]
    }
)

In [ ]:
print(response.content)

Event Title: Julia and Alex's Artful Nature Wedding

Audience: Family, friends, and loved ones of Julia and Alex, as well as art and nature enthusiasts.

Budget: $30,000

Venue: Central Park, New York City

Catering Options: 
- Organic and locally sourced menu options
- Vegetarian and vegan options
- Artfully presented dishes
- Champagne toast and signature cocktails

Entertainment:
- Live acoustic music during the ceremony
- DJ for the reception
- Interactive art stations for guests to create their own masterpieces
- Nature-inspired photo booth

Logistics:
- Ceremony and reception to be held in a secluded area of Central Park, surrounded by lush greenery and blooming flowers
- Tents and seating to be set up for guests
- Art installations and sculptures to be placed around the venue
- Transportation for guests to and from the park
- Permits and permissions for the event in Central Park

Potential Vendors:
- Catering: Farm-to-table catering company
- Music: Local acoustic musician and D

## View Trace in Langfuse

Now we can see that the trace incl. the prompt template have been logged to Langfuse

(TODO: Replace with CleanShot screenshot)

## Iterate on prompt in Langfuse
We can now continue adapting our prompt template in the Langfuse UI and continuously update the prompt template in our Langchain application via the script above.